# Drug Discovery Project

## DATASETS:
(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In [28]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

RDKit WARNING: [13:54:48] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')
#df = pd.read_csv('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv', index_col=0)

In [4]:
#df.head()
list(path.iterdir())

[PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/RdkitDescriptors.py'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_1024.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL240_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL244_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL219_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978_cl.csv')]

# RdkitDescriptors

In [5]:
# function for returning fingerprint from a specific smile.

def fp(smile, diam = 2, bits = 1024):

    mol = Chem.MolFromSmiles(smile)
    Chem.SanitizeMol(mol)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, diam, nBits = bits) 
    return fp

In [6]:
#ECFP4
#Generated Circular fingerprints hashed into n bits length vectors.

def ECFP(ifile, ofile, diam, bits):
    
    print(f"Making fingerprints for file: {ifile}")
    df = pd.read_csv(ifile)
    
    df.insert(2, "ECFP4_", df.SMILES.apply(fp))
    
    df[[f"ECFP4_{i+1}" for i in range(len(df.ECFP4_[0]))]] = df.ECFP4_.to_list()
    
    df.drop("ECFP4_", axis = 1, inplace = True)
    
    
    df.to_csv(path/ofile, index = None)
    return df

# Run the functions on a file from dataset and store the results

In [7]:
dataset='CHEMBL205_cl'

In [8]:
#ECFP(path/f'{dataset}.csv', f'./{dataset}_ecfp_1024.csv', 2, 1024)

In [9]:
df = pd.read_csv(path/f'{dataset}_ecfp_1024.csv')

In [10]:
df.head()

,CID,SMILES,Activity,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,...,ECFP4_1015,ECFP4_1016,ECFP4_1017,ECFP4_1018,ECFP4_1019,ECFP4_1020,ECFP4_1021,ECFP4_1022,ECFP4_1023,ECFP4_1024
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split data

In [11]:
# Split into X,y
X, y = df.drop(["CID", "SMILES", "Activity"], axis=1), df["Activity"]

In [12]:
# check info of dataframe
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 140.2 MB


In [13]:
# y is a pandas series
y.head(), y.size, type(y)

(0    1
 1    1
 2    1
 3    1
 4    1
 Name: Activity, dtype: int64,
 17941,
 pandas.core.series.Series)

# Train test split

In [14]:
from sklearn.model_selection import train_test_split, KFold

In [15]:
# regular train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

# 5-Fold Cross Validation

In [16]:
#5-fold
kf = KFold(n_splits=5, shuffle=True, random_state=999)

In [17]:
# append to a list / could write to csv file to keep integrity
X_train_list, X_valid_list, y_train_list, y_valid_list = [], [], [], []

for train_index, valid_index in kf.split(X_train):
    X_train_list.append(X_train.iloc[train_index])
    X_valid_list.append(X_train.iloc[valid_index])
    y_train_list.append(y_train.iloc[train_index])
    y_valid_list.append(y_train.iloc[valid_index])
    
   

In [18]:
y_train_list[0].head()

14960    0
16280    0
16068    0
13692    0
14043    0
Name: Activity, dtype: int64

In [19]:
X_train_list[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8611 entries, 14960 to 10114
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 67.3 MB


In [20]:
# TODO: add splits to csv file

# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [23]:
# train method for Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators=5, criterion='gini', max_features='log2'):

    
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, min_samples_split=2, max_features=max_features, 
                               max_leaf_nodes=None,bootstrap=False,oob_score=False, n_jobs=-1, random_state=69)
    
    rf.fit(X_train,y_train)
    y_pred= rf.predict(X_test)
    y_pred_prob=rf.predict_proba(X_test)
    
    temp=[]
    for j in range(len(y_pred_prob)):
        temp.append(y_pred_prob[j][1])
    auc=roc_auc_score(np.array(y_test),np.array(temp))
    acc2=accuracy_score(y_test,y_pred)
    mcc=matthews_corrcoef(y_test,y_pred)
    Recall=recall_score(y_test, y_pred,pos_label=1)
    Precision=precision_score(y_test, y_pred,pos_label=1)
    F1_score=f1_score(y_test, y_pred,pos_label=1)

    return auc,acc2,mcc,Recall,Precision,F1_score

In [24]:


# Train and print scores for each split
for i in range(0,5):
    
    print(f'Cross validation iteration: {i + 1}')
    
    X_train = X_train_list[i]
    X_valid = X_valid_list[i]
    y_train = y_train_list[i]
    y_valid = y_valid_list[i]
    
    auc,acc2,mcc,Recall,Precision,F1_score = train_rf(X_train, X_valid, y_train, y_valid, 
                                                      n_estimators=100, criterion='entropy', max_features='sqrt')
    
    print(f'AUC score:{auc}')
    print(f'Accuracy: {acc2}')
    print(f'Matthews: {mcc}')
    print(f'Recall: {Recall}')
    print(f'Precision: {Precision}')
    print(f'F1 score: {F1_score}')
    print()

Cross validation iteration: 1
AUC score:0.9887427620260744
Accuracy: 0.9730608453320948
Matthews: 0.8415714713449818
Recall: 0.8564356435643564
Precision: 0.8564356435643564
F1 score: 0.8564356435643564

Cross validation iteration: 2
AUC score:0.9876712143359863
Accuracy: 0.9781699953553181
Matthews: 0.8608807076715227
Recall: 0.8563829787234043
Precision: 0.8895027624309392
F1 score: 0.8726287262872627

Cross validation iteration: 3
AUC score:0.9909322903664254
Accuracy: 0.9712029725963771
Matthews: 0.8122047591950275
Recall: 0.8142076502732241
Precision: 0.8418079096045198
F1 score: 0.8277777777777778

Cross validation iteration: 4
AUC score:0.9754214676396435
Accuracy: 0.9693450998606595
Matthews: 0.8300350847464447
Recall: 0.7964601769911505
Precision: 0.9
F1 score: 0.8450704225352113

Cross validation iteration: 5
AUC score:0.9912994366040052
Accuracy: 0.9753717472118959
Matthews: 0.8390326671715927
Recall: 0.8406593406593407
Precision: 0.864406779661017
F1 score: 0.85236768802228